In [1]:
import json
import glob
import os

images = glob.glob('/data/wenhu/V2PE/dataset/image/long_mr/train/*')
images = [os.path.basename(image) for image in images]
images = set(images)
print(len(images))

1812251


In [ ]:

with open('/data/wenhu/V2PE/dataset/train_long_mr_image_retrieval_16_32.jsonl', 'r') as f:
    for line in f:
        entry = json.loads(line)
        matched = set(entry['image']) & images
        if len(matched) == len(entry['image']):
            print('All found')

In [5]:
from datasets import load_dataset
import json
import os
import random

dataset = load_dataset('TIGER-Lab/VisualWebInstruct', 'reference')

dict_template = [
    'Answer the above question. Conclude your final answers as a dictionary at the end of the response after ###',
    'Think and answer the question. Conclude your final answers as a dictionary at the end of the response after ###',
    'Think before answering the question. Conclude your final answers as a dictionary at the end of the response after ###',
    'Please answer the question above. Conclude your final answers as a dictionary at the end of the response after ###',
]

template = [
    'Think step by step to answer the question. Conclude your final answer at the end of the response after ###',
    'Think very deeply to answer the question. Conclude your final answer at the end of the response after ###',
    'Think carefully first before answering the question. Conclude your final answer at the end of the response after ###',
    'Please think step by step and then answer the question. Conclude your final answer at the end of the response after ###',
    'First reason and then answer the question. Conclude your final answer at the end of the response after ###',
    'Lay out your reasoning before answering the question. Conclude your final answer at the end of the response after ###',
]

count = 0
with open('dataset/train_visual_webinstruct.jsonl', 'w') as f:
    for d in dataset['train']:
        image = d['image']
        if not os.path.exists(f'dataset/image/visual_webinstruct/train/{d["idx"]}.png'):
            image.save(f'dataset/image/visual_webinstruct/train/{d["idx"]}.png')

        if len(d['short_answer']) > 50:
            prompt = ''
        else:
            if '{' in d['short_answer'] or '}' in d['short_answer']:
                prompt = random.choice(dict_template)
            else:
                prompt = random.choice(template)
            d['answer'] = d['answer'] + '\n' + '### ' + d['short_answer']

        new_prompt = '<image>' + ' ' + d['question'] + '\n' + prompt
        new_prompt = new_prompt.strip('\n').strip('\n').strip('\n').strip()

        # building prompt
        tmp = {
            'id': count,
            'image': [f'{d["idx"]}.png'],
            'conversations': [
                {'from': 'human', 'value': new_prompt},
                {'from': 'gpt', 'value': d['answer']}
            ]
        }
        f.write(json.dumps(tmp) + '\n')

In [6]:
from datasets import load_dataset
import json
import random

dataset = load_dataset('TIGER-Lab/VisualWebInstruct', 'LongCoT')

dict_template = [
    'Think step by step to answer the question. Conclude your final answers as a dictionary at the end of the response after ###',
    'Think very deeply to answer the question. Conclude your final answers as a dictionary at the end of the response after ###',
    'Think carefully first before answering the question. Conclude your final answers as a dictionary at the end of the response after ###',
    'Please think step by step and then answer the question. Conclude your final answers as a dictionary at the end of the response after ###',
    'First reason and then answer the question. Conclude your final answers as a dictionary at the end of the response after ###',
    'Lay out your reasoning before answering the question. Conclude your final answers as a dictionary at the end of the response after ###',
]

template = [
    'Think step by step to answer the question. Conclude your final answer at the end of the response after ###',
    'Think very deeply to answer the question. Conclude your final answer at the end of the response after ###',
    'Think carefully first before answering the question. Conclude your final answer at the end of the response after ###',
    'Please think step by step and then answer the question. Conclude your final answer at the end of the response after ###',
    'First reason and then answer the question. Conclude your final answer at the end of the response after ###',
    'Lay out your reasoning before answering the question. Conclude your final answer at the end of the response after ###',
]

with open('dataset/train_visual_webinstruct_long.jsonl', 'w') as f:
    for d in dataset['train']:

        if len(d['short_answer']) > 50:
            prompt = ''
        elif '\\boxed{' in d['answer']:
            prompt = 'Think step by step to derive your answer and put it in \\boxed{ANSWER}.'
        else:
            if '{' in d['short_answer'] or '}' in d['short_answer']:
                prompt = random.choice(dict_template)
            else:
                prompt = random.choice(template)
            d['answer'] = d['answer'] + '\n' + '### ' + d['short_answer']

        if '<image>' not in d['question']:
            new_prompt = '<image>' + ' ' + d['question'] + '\n' + prompt
        else:
            new_prompt = d['question'] + '\n' + prompt

        new_prompt = new_prompt.strip('\n').strip('\n').strip('\n').strip()

        if len(d['answer']) > 80:
            tmp = {
                'id': count,
                'image': [f'{d["idx"]}.png'],
                'conversations': [
                    {'from': 'human', 'value': new_prompt},
                    {'from': 'gpt', 'value': d['answer']}
                ]
            }
            f.write(json.dumps(tmp) + '\n')

Generating train split: 100%|██████████| 23694/23694 [00:00<00:00, 35484.26 examples/s]
